<a href="https://colab.research.google.com/github/sandhyaparna/CV-Training/blob/main/notebooks/OCR/02_click_based_ocr_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Apple Vision Framework

### Install packages

In [ ]:
# !pip install opencv-python pillow pyobjc-framework-Vision pyobjc jaro_winkler
# !pip install pandas
# !pip install pyobjc-framework-Vision
# !pip install apple-ocr
# !pip install pyobjc
# !pip install pyobjc-framework-Quartz
!pip install nbconvert[webpdf]
!pip install playwright
!playwright install chromium

### Import Packages

In [ ]:
import json
import os
import string
import time
import jaro
import cv2
import Quartz
import Vision
from Cocoa import NSURL
from IPython.display import Image, display
from PIL import Image, ImageDraw, ImageFont
from typing import List
from apple_vision_utils import extract_text, extract_text_bboxes, display_text_bboxes, get_text_at_click, highlight_click_text_bbox, display_click_text_bbox

In [ ]:
micro_ingredient_images = "/data/ingredients"

img_name = "Media (35).jpeg"
img_path = f"/data/ingredients/{img_name}"

image = Image.open(img_path)
# image = image.rotate(270, expand=True)
# image = ImageOps.exif_transpose(image)

display(image)

### Perform OCR on a single image

In [ ]:
extract_text(img_path)

#### Display an image with bounding box and text for OCR

In [ ]:
# display all the text and corresponding bboxes
display_text_bboxes(img_path)

In [ ]:
# highlights the text of interest in addition to displaying other text
highlight_click_text_bbox(img_path, click_x=500, click_y=600, padding=10)

In [ ]:
# The bbox is displayed only for the text of interest
display_click_text_bbox(img_path, click_x=500, click_y=600, padding=10)

## This code uses label dict instead of two label sets

In [ ]:
labels_dict = {
    "x": "A",
    "y": "B",
    "z": "C",
}

# Collect all direct matches (exact string matches) from processed_results
# using the keys in labels_dict. The values are the standardized labels.

# Get OCR text only around the click
raw_results = get_text_at_click(img_path, click_x=500, click_y=600, padding=10)
if not raw_results:
        print("No text detected at click location")

# Split text into all subphrases and preprocess
results = [word for text in raw_results for word in split_text(text)]
processed_results = {remove_after_special_char(word.lower()) for word in results if word.strip()}

# Direct match using labels_dict keys
direct_match = {labels_dict[result] for result in processed_results if result in labels_dict.keys()}

# If at least one direct match is found
if direct_match:
    print(direct_match)
    # Exactly one label found → success
    if len(direct_match) == 1:
        print("direct_match",direct_match)
    # More than one different label was matched → ambiguous result
    else:
        print("more than 2 labels found, retake image")

# If no direct match, try fuzzy matching
else:
    similarity_match = set()
    # Iterate through each result in the processed results
    for result in processed_results:
        # Compare it against each label in the primary label set
        for text, label  in labels_dict.items():
            # Compute the Jaro-Winkler similarity score between the result and the label
            jaro_winkler_score = jaro.jaro_winkler_metric(result, text)
            # If the similarity score is above a threshold (e.g., 0.8), consider it a match
            if jaro_winkler_score > 0.8:
                similarity_match.add(label)

    # If we found at least one fuzzy match
    if similarity_match:
        print(similarity_match)
        # Exactly one fuzzy match found → success
        if len(similarity_match) == 1:
            print("fuzzy_match",similarity_match)
        # Multiple fuzzy matches found → ambiguous
        else:
            print("more than 2 labels found, retake image")

    # Neither direct nor fuzzy match found
    else:
        print("no confident match found")
